In [ ]:
# install hugging face
! pip install transformers

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import torch, os, glob
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
import torchvision.models as models
import torchvision.transforms as tt
import torch.optim as optim
import cv2
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from transformers import AutoImageProcessor, ViTMAEModel
import os
import random
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# mounting on google drive
from google.colab import drive
drive.mount('/content/drive/')

In [4]:
# unzip UKT dataset part1
! tar -xzf drive/MyDrive/ColabNotebooks/part1.tar.gz

In [5]:
# define dataset

# split data into training and testing
all_paths = glob.glob('part1' + "/*.jpg")
random.shuffle(all_paths)
# all_paths = all_paths[:35]
train_path_len = int(0.92*len(all_paths))
train_paths = all_paths[:train_path_len]
valid_paths = all_paths[train_path_len:]

class UTKDataset(Dataset):
    def __init__(self, train=True, transform=None):
        self.train = train
        self.transform = transform
        file_list = train_paths if self.train else valid_paths
        self.data = []
        for f in file_list:
          path = f.split('/')[1]
          age = path.split('_')[0]
          gender = path.split('_')[1]
          if int(gender) not in [0, 1]:
            continue
          self.data.append([f, age, gender])

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        f, age, gender = self.data[idx]
        img = cv2.imread(f)
        if self.transform:
          img = self.transform(img)
        torch.reshape(img, (3, 224, -1))
        return img, age, gender

In [6]:
# apply data augmantation on both datasets

train_transform =transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(),
    transforms.RandomRotation(60),
    transforms.Resize((224,224))
    ])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224))
    ])

traindataset = UTKDataset(transform=train_transform)
train_dataloader = DataLoader(traindataset, batch_size=32, shuffle=True, drop_last=True)
testdataset = UTKDataset(train=False, transform=test_transform)
test_dataloader = DataLoader(testdataset, batch_size=32, shuffle=False, drop_last=False)

In [7]:
# define model

class MultiTaskModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = ViTMAEModel.from_pretrained("facebook/vit-mae-base")
        self.fc1 = nn.Linear(38400, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 128)

        self.age = nn.Linear(128, 1)
        self.gender = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.8)
        # self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.backbone(x).last_hidden_state
        x = torch.flatten(x, start_dim=1)
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.dropout(nn.functional.relu(self.fc2(x)))
        x = self.dropout(nn.functional.relu(self.fc3(x)))
        age_output = self.age(x)
        gender_output = self.gender(x)

        return [age_output, gender_output]

In [8]:
use_cuda = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 30
learning_rate = 1e-2

model = MultiTaskModel()
model = model.to(device)

optimizer = optim.Adam(model.parameters(), learning_rate, weight_decay=1e-5)
gender_loss_layer = nn.CrossEntropyLoss().to(device)
age_loss_layer = nn.MSELoss().to(device)

In [9]:
# Validation
val_age_loss_list = []
val_gender_loss_list = []

def test(model, test_loader):
    model.eval()
    total = len(test_loader.dataset)
    loss_item = 0
    age_loss_item = 0
    gender_loss_item = 0

    with torch.no_grad():
        for data in tqdm(test_loader):
            images, age, gender = data
            if use_cuda:
                images = images.cuda()
                age = np.array(age).astype(int)
                age = torch.from_numpy(age)
                age = age.to(torch.float32)
                age = age.cuda()

                gender = np.array(gender).astype(int)
                gender = torch.from_numpy(gender)
                gender = gender.cuda()

            age_out, gender_out = model(images)

            age_out = age_out.to(torch.float32)
            age_out = torch.squeeze(age_out)

            age_loss = age_loss_layer(age_out, age)
            gender_loss = gender_loss_layer(gender_out, gender)

            loss = age_loss/100*0.7 + gender_loss*3

            loss_item += loss.item()
            age_loss_item += age_loss.item()
            gender_loss_item += gender_loss.item()


        loss = loss_item / total
        age_loss = age_loss_item / total
        gender_loss = gender_loss_item / total
        val_age_loss_list.append(age_loss)
        val_gender_loss_list.append(gender_loss)

        print(f'Valid_Age_loss: {age_loss}, Valid_Gender_loss: {gender_loss}, Valid_Loss: {loss}')

In [ ]:
# Training

age_loss_list = []
gender_loss_list = []
total_loss_list = []

for epoch in range(1, epochs + 1):
    age_loss_epoch = 0
    gender_loss_epoch = 0
    if epoch % 2 == 0:
        learning_rate = learning_rate * 0.5
        optimizer = optim.Adam(model.parameters(), learning_rate, weight_decay=1e-5)
    model.train()

    correct = 0
    total = len(train_dataloader.dataset)
    loss_item = 0
    age_loss_item = 0
    gender_loss_item = 0

    for data in tqdm(train_dataloader):
        optimizer.zero_grad()

        images, age, gender = data
        if use_cuda:
            images = images.cuda()
            age = np.array(age).astype(int)
            age = torch.from_numpy(age)
            age = age.to(torch.float32)
            age = age.cuda()

            gender = np.array(gender).astype(int)
            gender = torch.from_numpy(gender)
            gender = gender.cuda()

        age_out, gender_out = model(images)

        age_out = age_out.to(torch.float32)
        age_out = torch.squeeze(age_out)

        age_loss = age_loss_layer(age_out, age)
        gender_loss = gender_loss_layer(gender_out, gender)

        loss = age_loss/100*0.7 + gender_loss*3

        loss.backward()
        optimizer.step()

        loss_item += loss.item()
        age_loss_item += age_loss.item()
        gender_loss_item += gender_loss.item()


    loss = loss_item / total
    age_loss = age_loss_item / total
    gender_loss = gender_loss_item / total
    total_loss_list.append(loss)
    age_loss_list.append(age_loss)
    gender_loss_list.append(gender_loss)

    print(f'Train_Age_loss: {age_loss}, Train_Gender_loss: {gender_loss}, Train_Loss: {loss}')

    test(model, test_dataloader)

    torch.save(model, 'drive/MyDrive/ColabNotebooks/model.pt')

In [ ]:
def plot(age_train, gender_train, age_val, gender_val):
    figure, axis = plt.subplots(1, 2)
    axis[0].plot(age_train, label='Train')
    axis[0].plot(age_val, label='Valid')
    axis[0].set_xlabel('epoch')
    axis[0].set_ylabel('Age loss')
    axis[0].set_title("Age Training and Validation Loss")
    axis[1].plot(gender_train, label='Train')
    axis[1].plot(gender_val, label='Valid')
    axis[1].legend(['Train','Valid'])
    axis[1].set_xlabel('epoch')
    axis[1].set_ylabel('Gender loss')
    axis[1].set_title("Gedner Training and Validation Accuracy")
    plt.show()
    plt.savefig("graph.jpg")

plot(age_loss_list, gender_loss_list, val_age_loss_list, val_gender_loss_list)